In [8]:
import numpy as np
import csv
import cv2
import tqdm
import glob

In [27]:
def process_bb(size, bb):
    dh = 1./size[0]
    dw = 1./size[1]

    x, y, w, h = bb
    x = int(x)
    y = int(y)
    w = int(w)
    h = int(h)

    x2 = x+w
    y2 = y+h

    xc = (x2 - x)/2.0
    yc = (y2 - y)/2.0

    return xc*dw, yc*dh, w*dw, h*dh

In [12]:
def populate_image_ids(file_path):
    image_ids = open(file_path).read().strip().split()
    return image_ids

def create_test_train_lists(image_ids, split=0.5):
    # now we iterate over all list_objects
    # Train
    train_file = open("train.txt", "w+")

    i = 0
    while i in range(int(len(image_ids) * split)):
        full_path = '%s\n' % (image_ids[i])
        train_file.write(full_path)
        i += 1
    train_file.close()

    test_file = open("test.txt", "w+")
    while i < len(image_ids):
        full_path = '%s\n' % (image_ids[i])
        test_file.write(full_path)
        i += 1
    test_file.close

In [29]:
def write_to_files(annot_dict, location):
    # Here do a final run over the dictionary and save to txt files
    # Annotations output file
    
    # Here we write the image_locations to the all.txt file
    list_file = open("all.txt", "a+")
    for image_id in tqdm.tqdm(annot_dict.keys()):
        list_file.write(f"data/{location}/{image_id}.jpg\n")

        # Writing annotations to individual txt files
        out_path = f"{location}/{image_id}.txt"
        with open(out_path, 'w+') as out_file:
            # For annotations for a single image
            for ann in annot_dict[image_id]:
                # Write this annotation to the file
                out_file.write(" ".join([str(a) for a in ann]) + '\n')

    print("Done writing annotations to file")
    list_file.close()

In [34]:
from PIL import Image

In [38]:
def coco2darknet(paths, location = "coco"):
    annot_dict = dict()
    persons_count = 0
    # faces_count = 0
    
    for annotations_path in paths:
        # Reading annotations into a dictionary
        with open(annotations_path) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0

            for row in tqdm.tqdm(csv_reader):
                
#                 # Debugging
                if (line_count % 1000) == 0:
                    write_to_files(annot_dict, location)
                    
                    print("100 images were written to files")
                    
                    annot_dict = {}

                file_path = row[0]
                image_id = file_path.split('/')[-1].split('.')[0]


                image = cv2.imread(file_path)
                
#                 Use this instead
#                 image = Image.open(file_path)
#                 size = image.size[::-1]

                # Skip the image which we dont have in our images folder
                # Possibly because it wasnt downloaded during the download process
                if np.all(image) == None:
                    continue

                # The COCO bounding box format is [top left x position, top left y position, width, height].
                # Converting the bounding boxes to darkent format here
                size = image.shape
                xc, yc, w, h = process_bb(size, row[1:-1])
                class_id = int(row[-1])

                if image_id not in annot_dict.keys():
                    annot_dict[image_id] = [[class_id, xc, yc, w, h]]
                    line_count += 1

                    # # Running yolo on the current image
                    # # Saving annotations in the dictionary
                    # faces = infer(file_path)
                    # faces_count += len(faces)
                    # # left, top, width, height

                    # for face in faces:
                    #     face_x, face_y, face_w, face_h = process_bb(size, face)
                    #     face_ann = [1, face_x, face_y, face_w, face_h]
                    #     annot_dict[image_id].append(face_ann)
                else:
                    annot_dict[image_id].append([class_id, xc, yc, w, h])
                persons_count += 1
                
    print("Persons Count = ", persons_count)

In [39]:
coco2darknet(["annotations.csv", "val_annotations.csv"])

558it [00:07, 74.12it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'images/000000262404.jpg'

In [24]:
import random
random.seed(3)

In [40]:
image_ids = populate_image_ids("all.txt")

In [42]:
create_test_train_lists(image_ids, split=0.6)

In [43]:
len(image_ids)

63289